In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [111]:
datatype = 'quality'

In [112]:
if datatype == "train":
    path = '/data/public/MLA/VCHAMPS/measurements_train.csv'
elif datatype == "test":
    path = '/data/public/MLA/VCHAMPS-Test/measurements_test.csv'
else:
    path = '/data/public/MLA/VCHAMPS-Quality/measurements_qual.csv'

measurements = pd.read_csv(path).iloc[:,1:]
measurements

,Internalpatientid,Age at measurement,Measurement date,Measurement,Result numeric,Result textual,State
0,100012,52.872186,2002-05-15 18:48:46.0,Pulse,86.000000,NaN,New Mexico
1,100012,52.872186,2002-05-15 18:48:46.0,Weight,387.467147,NaN,New Mexico
2,100012,52.872186,2002-05-15 18:48:46.0,Blood pressure,NaN,135/81,New Mexico
3,100012,52.872186,2002-05-15 18:48:46.0,Respiratory rate,20.000000,NaN,New Mexico
4,100012,52.872186,2002-05-15 18:48:46.0,Pain,2.000000,NaN,New Mexico
...,...,...,...,...,...,...,...
1069996,99944,80.630428,2013-02-24 10:06:41.0,Blood pressure,NaN,116/60,Massachusetts
1069997,99944,80.630428,2013-02-24 10:06:41.0,Temperature,100.000000,NaN,Massachusetts
1069998,99944,80.630428,2013-02-24 10:06:41.0,Pulse oximetry,98.000000,NaN,Massachusetts
1069999,99944,80.630428,2013-02-24 10:06:41.0,Pulse,60.000000,NaN,Massachusetts


In [113]:
measurements['Measurement'].unique()

array(['Pulse', 'Weight', 'Blood pressure', 'Respiratory rate', 'Pain',
       'Temperature', 'Pulse oximetry', 'Circumference/girth', 'Height',
       'Central venous pressure'], dtype=object)

In [114]:
measurements = measurements.drop(columns = ['Result textual','State'])

In [115]:
unique_pid = pd.DataFrame({'Internalpatientid':measurements['Internalpatientid'].unique()})
unique_pid

,Internalpatientid
0,100012
1,100223
2,100229
3,100314
4,100399
...,...
989,99944
990,37010
991,6400
992,64295


In [116]:
heights = measurements[measurements['Measurement'] == 'Height'][['Internalpatientid','Result numeric']]
heights = heights.rename(columns = {'Result numeric':'Height'})
heights = heights.groupby('Internalpatientid').mean().reset_index(drop = False)
heights = unique_pid.merge(heights, how = 'left', on = 'Internalpatientid')
heights 

,Internalpatientid,Height
0,100012,71.419314
1,100223,66.599832
2,100229,71.972222
3,100314,71.700000
4,100399,69.625000
...,...,...
989,99944,65.333333
990,37010,63.000000
991,6400,75.000000
992,64295,74.000000


In [117]:
heights.isna().sum()

Internalpatientid     0
Height               16
dtype: int64

In [118]:
weights = measurements[measurements['Measurement'] == 'Weight'].drop(columns = ['Age at measurement', 'Measurement'])
weights = weights.rename(columns = {'Result numeric':'Weight'})
weights

,Internalpatientid,Measurement date,Weight
1,100012,2002-05-15 18:48:46.0,387.467147
9,100012,2002-10-12 01:01:26.0,404.000000
15,100012,2003-01-04 17:36:15.0,381.818977
17,100012,2004-01-17 04:10:26.0,391.000000
19,100012,2004-04-25 02:24:39.0,374.000000
...,...,...,...
1069909,99870,1999-03-25 17:06:17.0,204.000000
1069916,99870,2007-09-23 12:10:03.0,191.719938
1069936,99870,2008-12-27 02:33:31.0,191.638681
1069961,99941,2015-10-20 14:18:41.0,213.361541


In [119]:
pulse_oximetries = measurements[measurements['Measurement'] == 'Pulse oximetry']\
    .drop(columns = ['Age at measurement', 'Measurement'])
pulse_oximetries = pulse_oximetries.rename(columns = {'Result numeric':'Pulse oximetry'})
pulse_oximetries

,Internalpatientid,Measurement date,Pulse oximetry
51,100012,2009-01-07 07:39:38.0,100.0
60,100012,2016-12-31 19:19:21.0,94.0
61,100012,2017-12-27 21:27:23.0,91.0
80,100012,2004-03-01 11:27:14.0,89.0
108,100012,2018-12-17 09:21:52.0,98.0
...,...,...,...
1069973,99941,2015-11-28 10:37:14.0,92.0
1069975,99941,2015-11-30 12:42:13.0,98.0
1069983,99941,2015-12-07 12:33:33.0,98.0
1069990,99941,2015-12-10 09:21:42.0,94.0


## For readmission
### Calculate mean, min, max, std for weights and Pulse oximetries

In [120]:
if datatype == "train":
    path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
elif datatype == "test":
    path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
else:
    path = '/home/hassan/lily/MLA/FDA/inpatient_simple_quality.csv'
    
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,CVD,unique_admitting_specialty,unique_discharging_specialty,DOMICILIARY,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,67,1,0.18,1,0,0,0,1,1,0,...,1.00,1,0,0,0,0,0,0.00,2019-04-29 22:39:20.0,0
1,200,4,17.90,2,0,0,0,4,3,0,...,0.25,1,0,0,2,0,0,130.54,2019-01-31 22:59:02.0,1
2,291,7,79.68,2,0,3,1,3,2,0,...,2.33,0,0,0,7,0,0,-6.93,2006-07-08 13:56:24.0,1
3,330,3,17.38,1,0,1,1,2,2,0,...,3.00,1,0,0,2,0,0,993.85,2019-06-28 14:50:58.0,0
4,351,3,22.14,1,0,3,1,2,2,0,...,3.00,0,2,0,0,1,1,77.26,2023-07-10 21:33:42.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,166881,1,2.52,1,0,1,1,1,1,0,...,1.00,1,0,0,0,0,0,410.96,2001-01-09 21:07:54.0,0
628,167102,3,10.25,1,0,0,0,3,1,0,...,0.38,2,0,0,1,0,0,2026.77,2011-11-27 12:57:02.0,0
629,167404,1,11.63,1,0,1,1,1,1,0,...,1.00,0,0,0,1,0,0,0.00,2004-01-30 11:12:01.0,0
630,167917,4,10.50,1,0,1,1,4,2,0,...,1.33,3,0,0,1,0,1,1354.04,2016-08-07 11:25:36.0,0


In [121]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,Discharge date
0,67,2019-04-29 22:39:20.0
1,200,2019-01-31 22:59:02.0
2,291,2006-07-08 13:56:24.0
3,330,2019-06-28 14:50:58.0
4,351,2023-07-10 21:33:42.0
...,...,...
627,166881,2001-01-09 21:07:54.0
628,167102,2011-11-27 12:57:02.0
629,167404,2004-01-30 11:12:01.0
630,167917,2016-08-07 11:25:36.0


In [122]:
# Filter procedure date that's before discharge date
inpatient_weight = inpatient_date.merge(weights, how = 'left', on = 'Internalpatientid')
inpatient_weight = inpatient_weight.loc[inpatient_weight['Measurement date'] <= inpatient_weight['Discharge date']]
inpatient_weight = inpatient_weight.groupby('Internalpatientid').mean().reset_index(drop = False)
inpatient_weight

/tmp/ipykernel_353911/1155935740.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  inpatient_weight = inpatient_weight.groupby('Internalpatientid').mean().reset_index(drop = False)


,Internalpatientid,Weight
0,67,188.761513
1,200,176.164106
2,291,163.106281
3,330,279.201471
4,351,240.791052
...,...,...
619,166881,160.029474
620,167102,205.170035
621,167404,135.054080
622,167917,223.816299


In [123]:
inpatient_pulse_ox = inpatient_date.merge(pulse_oximetries, how = 'left', on = 'Internalpatientid')
inpatient_pulse_ox = inpatient_pulse_ox.loc[inpatient_pulse_ox['Measurement date'] <= inpatient_pulse_ox['Discharge date']]
inpatient_pulse_ox['Pulse oximetry min'] = inpatient_pulse_ox['Pulse oximetry'].groupby(inpatient_pulse_ox['Internalpatientid']).transform('min')
inpatient_pulse_ox['Pulse oximetry max'] = inpatient_pulse_ox['Pulse oximetry'].groupby(inpatient_pulse_ox['Internalpatientid']).transform('max')
inpatient_pulse_ox['Pulse oximetry mean'] = inpatient_pulse_ox['Pulse oximetry'].groupby(inpatient_pulse_ox['Internalpatientid']).transform('mean')
inpatient_pulse_ox['Pulse oximetry std'] = inpatient_pulse_ox['Pulse oximetry'].groupby(inpatient_pulse_ox['Internalpatientid']).transform('std')
inpatient_pulse_ox = inpatient_pulse_ox.drop(columns = ['Discharge date','Measurement date','Pulse oximetry'])
inpatient_pulse_ox = inpatient_pulse_ox.groupby('Internalpatientid').mean().reset_index(drop = False)
inpatient_pulse_ox

,Internalpatientid,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,91.0,103.0,97.700000,3.097830
1,200,2.0,101.0,92.145161,9.032636
2,330,88.0,102.0,94.637363,3.507188
3,351,87.0,101.0,94.230769,2.898570
4,444,92.0,103.0,96.444444,2.791930
...,...,...,...,...,...
539,166603,90.0,102.0,95.261905,3.261449
540,167102,91.0,102.0,96.535714,2.937398
541,167404,94.0,104.0,98.000000,3.505098
542,167917,90.0,101.0,96.333333,2.777825


In [124]:
df = inpatient_date.merge(heights, how = 'left', on = 'Internalpatientid')
df = df.merge(inpatient_weight,how = 'left', on = 'Internalpatientid')
df = df.merge(inpatient_pulse_ox, how = 'left', on = 'Internalpatientid')
df = df.drop(columns = 'Discharge date')
df

,Internalpatientid,Height,Weight,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,69.923239,188.761513,91.0,103.0,97.700000,3.097830
1,200,65.266667,176.164106,2.0,101.0,92.145161,9.032636
2,291,68.142857,163.106281,NaN,NaN,NaN,NaN
3,330,66.861628,279.201471,88.0,102.0,94.637363,3.507188
4,351,74.589345,240.791052,87.0,101.0,94.230769,2.898570
...,...,...,...,...,...,...,...
627,166881,63.333333,160.029474,NaN,NaN,NaN,NaN
628,167102,75.970930,205.170035,91.0,102.0,96.535714,2.937398
629,167404,68.600000,135.054080,94.0,104.0,98.000000,3.505098
630,167917,71.277778,223.816299,90.0,101.0,96.333333,2.777825


In [125]:
if datatype == "train": df.to_csv("/home/daisy/FDA_Dataset/measurements_admission_train.csv")
if datatype == "test": df.to_csv("/home/daisy/FDA_Dataset/measurements_admission_test.csv")
if datatype == "quality": df.to_csv("/home/daisy/FDA_Dataset/measurements_admission_quality.csv")

## All Cause Mortality
### Calculate mean, min, max, std for weights and Pulse oximetries

In [126]:
if datatype == "train":
    path = '/home/hassan/lily/MLA/FDA/outpatient_mortality.csv'
elif datatype == "test":
    path = '/home/hassan/lily/MLA/FDA/outpatient_mortality_test.csv'
else:
    path = '/home/hassan/lily/MLA/FDA/outpatient_mortality_quality.csv'
    
outpatient = pd.read_csv(path, index_col=0)
outpatient

,Internalpatientid,num_visits,CVD,last_visit_date,Age 00-20,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,age_mean,age_std,freq
0,67,313,0,2020-11-11 16:00:31.0,0,0,313,0,0,0,53.523105,3.128784,28.45
0,200,1872,1,2023-02-13 10:31:19.0,0,0,0,741,1131,0,80.476890,4.781990,81.39
0,291,234,0,2007-03-26 14:15:45.0,0,0,0,0,234,0,81.506487,1.264804,58.50
0,330,932,1,2024-03-01 00:19:37.0,0,0,51,881,0,0,69.622150,4.492263,40.52
0,351,773,1,2024-06-04 14:45:34.0,0,0,0,0,773,0,85.033989,0.541659,154.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,167907,181,0,2013-11-14 23:44:48.0,0,0,0,31,150,0,84.030741,3.355429,13.92
0,167917,442,1,2022-01-27 05:25:51.0,0,0,442,0,0,0,45.647418,2.660564,40.18
0,168008,776,1,2013-12-03 23:38:48.0,0,0,0,776,0,0,71.853738,1.572813,86.22
0,168496,151,1,2023-02-03 22:36:15.0,0,0,0,9,142,0,92.361910,6.061224,6.57


In [127]:
outpatient_date = outpatient[['Internalpatientid','last_visit_date']]
outpatient_date = outpatient_date.drop_duplicates(keep = 'first')
outpatient_date

,Internalpatientid,last_visit_date
0,67,2020-11-11 16:00:31.0
0,200,2023-02-13 10:31:19.0
0,291,2007-03-26 14:15:45.0
0,330,2024-03-01 00:19:37.0
0,351,2024-06-04 14:45:34.0
...,...,...
0,167907,2013-11-14 23:44:48.0
0,167917,2022-01-27 05:25:51.0
0,168008,2013-12-03 23:38:48.0
0,168496,2023-02-03 22:36:15.0


In [128]:
# Filter procedure date that's before discharge date
outpatient_weight = outpatient_date.merge(weights, how = 'left', on = 'Internalpatientid')
outpatient_weight = outpatient_weight.loc[outpatient_weight['Measurement date'] <= outpatient_weight['last_visit_date']]
outpatient_weight = outpatient_weight.groupby('Internalpatientid').mean().reset_index(drop = False)
outpatient_weight

/tmp/ipykernel_353911/1451451851.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  outpatient_weight = outpatient_weight.groupby('Internalpatientid').mean().reset_index(drop = False)


,Internalpatientid,Weight
0,67,191.533739
1,200,176.545136
2,291,163.106281
3,330,275.040771
4,351,239.775651
...,...,...
982,167907,179.737913
983,167917,223.853588
984,168008,172.884283
985,168496,146.344572


In [129]:
outpatient_pulse_ox = outpatient_date.merge(pulse_oximetries, how = 'left', on = 'Internalpatientid')
outpatient_pulse_ox = outpatient_pulse_ox.loc[outpatient_pulse_ox['Measurement date'] <= outpatient_pulse_ox['last_visit_date']]
outpatient_pulse_ox['Pulse oximetry min'] = outpatient_pulse_ox['Pulse oximetry'].groupby(outpatient_pulse_ox['Internalpatientid']).transform('min')
outpatient_pulse_ox['Pulse oximetry max'] = outpatient_pulse_ox['Pulse oximetry'].groupby(outpatient_pulse_ox['Internalpatientid']).transform('max')
outpatient_pulse_ox['Pulse oximetry mean'] = outpatient_pulse_ox['Pulse oximetry'].groupby(outpatient_pulse_ox['Internalpatientid']).transform('mean')
outpatient_pulse_ox['Pulse oximetry std'] = outpatient_pulse_ox['Pulse oximetry'].groupby(outpatient_pulse_ox['Internalpatientid']).transform('std')
outpatient_pulse_ox = outpatient_pulse_ox.drop(columns = ['last_visit_date','Measurement date','Pulse oximetry'])
outpatient_pulse_ox = outpatient_pulse_ox.groupby('Internalpatientid').mean().reset_index(drop = False)
outpatient_pulse_ox

,Internalpatientid,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,91.0,105.0,97.972973,3.218751
1,200,2.0,101.0,92.309942,8.150270
2,330,82.0,102.0,94.524691,3.514302
3,351,87.0,101.0,94.027778,3.128807
4,444,88.0,103.0,96.830508,3.108124
...,...,...,...,...,...
822,167907,93.0,103.0,98.000000,4.472136
823,167917,90.0,102.0,96.537313,2.940462
824,168008,81.0,105.0,98.785441,3.524630
825,168496,0.0,100.0,84.687500,30.394558


In [130]:
df_allcause = outpatient_date.merge(heights, how = 'left', on = 'Internalpatientid')
df_allcause = df_allcause.merge(outpatient_weight,how = 'left', on = 'Internalpatientid')
df_allcause = df_allcause.merge(outpatient_pulse_ox, how = 'left', on = 'Internalpatientid')
df_allcause = df_allcause.drop(columns = 'last_visit_date')
df_allcause

,Internalpatientid,Height,Weight,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,69.923239,191.533739,91.0,105.0,97.972973,3.218751
1,200,65.266667,176.545136,2.0,101.0,92.309942,8.150270
2,291,68.142857,163.106281,NaN,NaN,NaN,NaN
3,330,66.861628,275.040771,82.0,102.0,94.524691,3.514302
4,351,74.589345,239.775651,87.0,101.0,94.027778,3.128807
...,...,...,...,...,...,...,...
995,167907,68.013691,179.737913,93.0,103.0,98.000000,4.472136
996,167917,71.277778,223.853588,90.0,102.0,96.537313,2.940462
997,168008,68.850000,172.884283,81.0,105.0,98.785441,3.524630
998,168496,63.909091,146.344572,0.0,100.0,84.687500,30.394558


In [131]:
if datatype == "train": df.to_csv("/home/daisy/FDA_Dataset/measurements_allcause_mortality_train.csv")
if datatype == "test": df.to_csv("/home/daisy/FDA_Dataset/measurements_allcause_mortality_test.csv")
if datatype == "quality": df.to_csv("/home/daisy/FDA_Dataset/measurements_allcause_mortality_quality.csv")

## CVD Mortality
### Calculate mean, min, max, std for weights and Pulse oximetries

In [132]:
cvd_weights = weights.groupby('Internalpatientid').mean().reset_index(drop = False)
cvd_weights

/tmp/ipykernel_353911/3277473481.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cvd_weights = weights.groupby('Internalpatientid').mean().reset_index(drop = False)


,Internalpatientid,Weight
0,67,191.533739
1,200,176.545136
2,291,163.106281
3,330,275.040771
4,351,239.775651
...,...,...
983,167907,179.737913
984,167917,223.853588
985,168008,172.884283
986,168496,146.344572


In [133]:
cvd_pulse_ox = pulse_oximetries
cvd_pulse_ox['Pulse oximetry min'] = cvd_pulse_ox['Pulse oximetry'].groupby(cvd_pulse_ox['Internalpatientid']).transform('min')
cvd_pulse_ox['Pulse oximetry max'] = cvd_pulse_ox['Pulse oximetry'].groupby(cvd_pulse_ox['Internalpatientid']).transform('max')
cvd_pulse_ox['Pulse oximetry mean'] = cvd_pulse_ox['Pulse oximetry'].groupby(cvd_pulse_ox['Internalpatientid']).transform('mean')
cvd_pulse_ox['Pulse oximetry std'] = cvd_pulse_ox['Pulse oximetry'].groupby(cvd_pulse_ox['Internalpatientid']).transform('std')
cvd_pulse_ox = cvd_pulse_ox.drop(columns = ['Measurement date','Pulse oximetry'])
cvd_pulse_ox = cvd_pulse_ox.groupby('Internalpatientid').mean().reset_index(drop = False)
cvd_pulse_ox

,Internalpatientid,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,91.0,105.0,97.972973,3.218751
1,200,2.0,101.0,92.309942,8.150270
2,330,82.0,102.0,94.524691,3.514302
3,351,87.0,101.0,94.027778,3.128807
4,444,88.0,103.0,96.830508,3.108124
...,...,...,...,...,...
823,167907,93.0,103.0,98.000000,4.472136
824,167917,90.0,102.0,96.537313,2.940462
825,168008,81.0,105.0,98.785441,3.524630
826,168496,0.0,100.0,84.687500,30.394558


In [134]:
df_cvd = heights.merge(cvd_weights, how = 'left', on = 'Internalpatientid')
df_cvd = df_cvd.merge(cvd_pulse_ox, how = 'left', on = 'Internalpatientid')
df_cvd = df_cvd.sort_values(by = 'Internalpatientid').reset_index(drop = True)
df_cvd 

,Internalpatientid,Height,Weight,Pulse oximetry min,Pulse oximetry max,Pulse oximetry mean,Pulse oximetry std
0,67,69.923239,191.533739,91.0,105.0,97.972973,3.218751
1,200,65.266667,176.545136,2.0,101.0,92.309942,8.150270
2,291,68.142857,163.106281,NaN,NaN,NaN,NaN
3,330,66.861628,275.040771,82.0,102.0,94.524691,3.514302
4,351,74.589345,239.775651,87.0,101.0,94.027778,3.128807
...,...,...,...,...,...,...,...
989,167907,68.013691,179.737913,93.0,103.0,98.000000,4.472136
990,167917,71.277778,223.853588,90.0,102.0,96.537313,2.940462
991,168008,68.850000,172.884283,81.0,105.0,98.785441,3.524630
992,168496,63.909091,146.344572,0.0,100.0,84.687500,30.394558


In [135]:
if datatype == "train": df.to_csv("/home/daisy/FDA_Dataset/measurements_cvd_mortality_train.csv")
if datatype == "test": df.to_csv("/home/daisy/FDA_Dataset/measurements_cvd_mortality_test.csv")
if datatype == "quality": df.to_csv("/home/daisy/FDA_Dataset/measurements_cvd_mortality_quality.csv")